# Add Memory to the `Arborist`

For more conversational applications it is possible to pass chat history to the `Arborist`. In this cookbook we simulate a trivial conversation in order to illustrate the use of `Memory`.


## Memory implementation

For our simulated conversation we implement a super simple version of the `Memory` abstraction which simply keeps an in-memory list of `ChatMessage`s. A real client would typically wrap a more powerful memory strategy (e.g. `llamaindex` long-short memory implementation).

In [1]:
from typing import List
from treelang.ai.memory import Memory, ChatMessage

class SimpleMemory(Memory):
    """A simple in-memory implementation of the Memory abstraction."""

    def __init__(self):
        self.messages: List[ChatMessage] = []

    async def add(self, messages: List[ChatMessage]) -> None:
        self.messages.extend(messages)

    async def get(self) -> List[ChatMessage]:
        return self.messages

    async def clear(self) -> None:
        self.messages = []

## The chat

For this super simple chat we re-use our trusty `MCP` calculator...

In [2]:
import asyncio
import os

# mcp components for stdio communication
from mcp import ClientSession, StdioServerParameters, stdio_client

# our helpful `Arborist` implementation
from treelang.ai.arborist import OpenAIArborist
from treelang.ai.provider import MCPToolProvider

async def main():
    # server parameters for running the calculator server
    path = os.path.join(os.getcwd(), "calculator.py")
    memory = SimpleMemory()
    server_params = StdioServerParameters(
        path=path, 
        command="python", args=[path], env=None
    )
    
    # establish a communication channel with the server
    async with stdio_client(server_params) as (read, write):
        # create a client session for interaction
        async with ClientSession(read, write) as session:
            # initialize the session
            await session.initialize()
            provider = MCPToolProvider(session)
            # create and configure the Arborist
            arborist = OpenAIArborist(
                model="gpt-4o-2024-11-20", 
                provider=provider, 
                memory=memory
            )
            # simulate a conversation
            query1 = "Add 2 and 3."
            response1 = await arborist.eval(query1)
            print("First response:", response1.content)

            # store the first interaction in memory
            await memory.add([
                ChatMessage(role="user", content=query1),
                ChatMessage(role="assistant", content=str(response1.content))
            ])

            # now ask a follow-up question!
            query2 = "Now multiply the result by 4."
            response2 = await arborist.eval(query2)
            print("Second response:", response2.content)

            # store the second interaction in memory
            await memory.add([
                ChatMessage(role="user", content=query2),
                ChatMessage(role="assistant", content=str(response2.content))
            ])

            query3 = "Instead of adding the first two numbers, what if we subtracted them?"
            response3 = await arborist.eval(query3)
            
            print("Third response:", response3.content)

            await memory.add([
                ChatMessage(role="user", content=query3),
                ChatMessage(role="assistant", content=str(response3.content))
            ])

            # we're done! Let's see the conversation history
            print("Conversation history:")

            for msg in await memory.get():
                print(f"{msg.role}: {msg.content}") 
            
            # finally, clear the memory
            await memory.clear()

## Main

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-5' coro=<main() running at /tmp/ipykernel_28629/3785840027.py:11>>

First response: 5.0
Second response: 20.0
Third response: -4.0
Conversation history:
user: Add 2 and 3.
assistant: 5.0
user: Now multiply the result by 4.
assistant: 20.0
user: Instead of adding the first two numbers, what if we subtracted them?
assistant: -4.0
